# test insert/fetch by `COPY`

In [1]:
import time
import io
import datetime
import pandas as pd
from opdb import models,utils,manage,opdb

## database info

In [2]:
hostname=''
port=''
dbname=''
username=''
passwd=''

In [3]:
url='postgresql://%s:%s@%s:%s/%s' % (username, passwd, hostname, port, dbname)

## test insert

In [4]:
n_record = 1000

### test `insert`

In [5]:
df = pd.DataFrame({'proposal_id': ['o%06d' % (n+1) for n in range(n_record)], 
                   'created_at': [datetime.datetime.now() for n in range(n_record)], 
                   'updated_at': [datetime.datetime.now() for n in range(n_record)]}
                 )
df[:5]

,proposal_id,created_at,updated_at
0,o000001,2021-07-13 11:09:29.358867,2021-07-13 11:09:29.359528
1,o000002,2021-07-13 11:09:29.358870,2021-07-13 11:09:29.359529
2,o000003,2021-07-13 11:09:29.358870,2021-07-13 11:09:29.359530
3,o000004,2021-07-13 11:09:29.358871,2021-07-13 11:09:29.359531
4,o000005,2021-07-13 11:09:29.358872,2021-07-13 11:09:29.359531


In [6]:
db = opdb.OpDB(hostname=hostname, port=port, dbname=dbname, username=username, passwd=passwd)
db.connect()
db.reset(full=True)
db.close()

In [7]:
start = time.time()

tab = 'proposal'
try:
    utils.insert(url, tab, df)
except:
    utils.update(url, tab, df)
    
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:248.8731997013092[sec]


### test `insert_by_copy`

In [8]:
data = io.StringIO()
for n in range(n_record):
    data.write('o%06d,' % (n+1))
    data.write('%s,' % (datetime.datetime.now()))
    data.write('%s\n' % (datetime.datetime.now()))
    
data.seek(0)

colnames = ('proposal_id',
            'created_at',
            'updated_at'
           )

In [9]:
db = opdb.OpDB(hostname=hostname, port=port, dbname=dbname, username=username, passwd=passwd)
db.connect()
db.reset(full=True)
db.close()

In [10]:
start = time.time()

tab = 'proposal'
try:
    utils.insert_by_copy(url, tab, data, colnames)
except:
    print('error...')    

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")    

elapsed_time:5.2995829582214355[sec]


## test fetch

In [11]:
tab = 'proposal'

In [12]:
colnames = ('proposal_id',
            'created_at',
            'updated_at'
           )

### test `fetch`

In [13]:
start = time.time()

df = utils.fetch_all(url, tab)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")    

elapsed_time:4.901716947555542[sec]


### test `fetch_by_copy`

In [14]:
start = time.time()

res = utils.fetch_by_copy(url, tab, colnames)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")    

elapsed_time:4.595918893814087[sec]
